In [1]:
from functions import *

In [1]:
from unsloth import FastModel,FastLanguageModel
import torch
from torch.nn.utils import clip_grad_value_
import torch.nn as nn
from peft import PeftModel
import gc
# from transformers.models.gemma3.modeling_gemma3 import Gemma3TextScaledWordEmbedding
from functions import *
model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    # model_name="unsloth/gemma-3-12b-pt",
    # model_name="unsloth/gemma-3-4b-pt",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
    resize_model_vocab=16,
)
model = model.base_model
model.lm_head.weight.requires_grad_(True);
gc.collect()
torch.cuda.empty_cache()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-04 07:57:07 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.642 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
model.lm_head.load_state_dict(torch.load('../Model/lm_heads_weights_pt.pth'))
model = PeftModel.from_pretrained(model, "../Model/merged_model_pt/",is_trainable=False)
model = FastLanguageModel.for_inference(model)

In [4]:
import json
output_path = '/home/zhenlan/Desktop/Projects/ARC2/Data/ARC-AGI-2-main/combined_data.json'
with open(output_path, 'r') as f:
    data = json.load(f)

In [5]:
gen = iter(data_gen(data,False,8600))
x, y = next(gen)
x, y = next(gen)

In [6]:
idx = torch.where(x[0] == 13)[0][-1].item() # last index of BOS
x, y = x[:,:idx+1], x[0][idx+1:] # x ends with BOS_Y (13), y starts from the next token

In [7]:
len(y)

421

In [8]:
y

tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0, 12,  7,  0,  0,  7,  7,  0,  0,  7,  7,  0,  0,  7,  7,  0,  0,
         7,  7,  0,  0,  7, 12,  0,  7,  7,  0,  0,  7,  7,  0,  0,  7,  7,  0,
         0,  7,  7,  0,  0,  7,  7,  0, 12,  0,  7,  7,  0,  0,  7,  7,  0,  0,
         7,  7,  0,  0,  7,  7,  0,  0,  7,  7,  0, 12,  7,  0,  0,  7,  7,  0,
         0,  7,  7,  0,  0,  7,  7,  0,  0,  7,  7,  0,  0,  7, 12,  7,  0,  0,
         7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,  0,  0,  7, 12,
         0,  7,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  7,
         7,  0, 12,  0,  7,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  7,  7,  0, 12,  7,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  7,  0,  0,  7, 12,  7,  0,  0,  7,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  7,  0,  0,  7, 12,  0,  7,  7,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 

In [10]:
# %debug
search = MinNLLSearcher(model)
search.dfs_generate(x)

Found EOS. Path NLL: 0.1276 | Path Len: 931
Found EOS. Path NLL: 0.1291 | Path Len: 931
Found EOS. Path NLL: 0.1288 | Path Len: 930
Found EOS. Path NLL: 0.1289 | Path Len: 931
Found EOS. Path NLL: 0.1308 | Path Len: 931
Found EOS. Path NLL: 0.1290 | Path Len: 931
Found EOS. Path NLL: 0.1306 | Path Len: 931
Found EOS. Path NLL: 0.1314 | Path Len: 931
Found EOS. Path NLL: 0.1292 | Path Len: 931
Found EOS. Path NLL: 0.1289 | Path Len: 930
Found EOS. Path NLL: 0.1317 | Path Len: 931
Found EOS. Path NLL: 0.1307 | Path Len: 931
Found EOS. Path NLL: 0.1307 | Path Len: 931
Found EOS. Path NLL: 0.1276 | Path Len: 931
Found EOS. Path NLL: 0.1276 | Path Len: 930
Found EOS. Path NLL: 0.1312 | Path Len: 931
Found EOS. Path NLL: 0.1301 | Path Len: 931
Found EOS. Path NLL: 0.1295 | Path Len: 931
Found EOS. Path NLL: 0.1311 | Path Len: 931
Found EOS. Path NLL: 0.1324 | Path Len: 931
Found EOS. Path NLL: 0.1298 | Path Len: 931
Found EOS. Path NLL: 0.1295 | Path Len: 930
Found EOS. Path NLL: 0.1320 | Pa

KeyboardInterrupt: 

Cumulative NLL

In [ ]:
class MinNLLSearcher(object):
    def __init__(self, model, max_depth: int = 31 * 30 + 1, brunch_factor: int = 3):
        """Initialize the searcher with a pre-trained model."""

        self.max_depth = max_depth
        self.model = model
        self.best_paths = []
        self.nlls = []
        self.min_nll = float('inf')
        self.brunch_factor = brunch_factor

    @staticmethod
    def check_equal_line_lengths(tensor):
        """ Check if all lines in a torch tensor have the same length when called at a line break position."""
        tensor = tensor[0]
        if tensor[-1].item() != 12: # only check if the last token is a line break
            return True
        idx = (tensor == 12).nonzero(as_tuple=True)[0]
        if len(idx) <= 1:
            return True
        return len(torch.unique(idx[1:] - idx[:-1])) == 1
        
    @torch.no_grad()
    def dfs_generate(self, current_ids, current_nll = 0, past_key_values = None, current_depth = 0):
        """Performs Depth-First Search to find the lowest NLL completion."""
        # current_ids is torch.Tensor of Shape: (1, seq_len)
        model = self.model
        max_depth = self.max_depth
        device = model.device
        # Safety check for recursion depth
        if current_depth > max_depth:
            return

        # Prepare inputs for the model
        if past_key_values is None:
            # First call, process the whole sequence
            input_ids_step = current_ids
        else:
            # Subsequent calls, only process the last token
            input_ids_step = current_ids[:, -1:]

        # Get logits and new KV cache
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            outputs = model(
                input_ids=input_ids_step,
                past_key_values=past_key_values,
                use_cache=True, # Crucial for efficiency
            )

        logits = outputs.logits.float() # Shape: (1, input_seq_len, vocab_size)
        new_past_key_values = outputs.past_key_values # Updated KV cache

        # Get logits for the *next* token prediction
        # Logits shape is (batch_size, sequence_length, vocab_size)
        next_token_logits = logits[:, -1, :]

        # Calculate log probabilities and negative log likelihoods
        log_probs = torch.log_softmax(next_token_logits, dim=-1)
        nlls = -log_probs # Shape: (1, vocab_size)

        # Sort potential next tokens by NLL (ascending)
        # We only need to explore the top `branching_factor` candidates
        sorted_nlls, sorted_indices = torch.sort(nlls.squeeze(), descending=False)
        sorted_nlls = sorted_nlls[:self.brunch_factor]
        sorted_indices = sorted_indices[:self.brunch_factor]
        
        # Iterate through the most promising next tokens
        for next_token_id, next_token_nll in zip(sorted_indices, sorted_nlls):
            next_token_id = next_token_id.item()
            next_token_nll = next_token_nll.item()

            potential_total_nll = current_nll + next_token_nll

            # --- Pruning ---
            if potential_total_nll >= self.min_nll:
                # If the current path's NLL is already worse than the best complete
                # sequence found so far, prune this branch.
                break

            
            if past_key_values is None: # first call
                next_ids = torch.tensor([[next_token_id]], dtype=torch.long, device=device)
            else: # Append the chosen token
                next_ids = torch.cat(
                    [current_ids, torch.tensor([[next_token_id]], dtype=torch.long, device=device)],
                    dim=1
                )
            
            if not self.check_equal_line_lengths(next_ids):
                # If the line lengths are not equal, prune this branch
                continue
            
            # --- Base Case: EOS token ---
            if next_token_id == 14 and next_ids[0][-2].item() == 12: # Line break followed by EOS
                print(f"Found EOS. Path NLL: {potential_total_nll:.4f} | Path Len: {current_depth}")
                self.best_paths.append(next_ids[0].tolist())
                self.nlls.append(potential_total_nll)
                self.min_nll = min(self.min_nll, potential_total_nll)
                # Continue searching other branches even after finding an EOS
                continue # Don't recurse further down this path

            # --- Recursive Step ---
            # Pass the `new_past_key_values` which contains the cache state *after*
            self.dfs_generate(current_ids=next_ids,
                              current_nll=potential_total_nll,
                              past_key_values=new_past_key_values,
                              current_depth=current_depth + 1,
                             )
        # del new_past_key_values, past_key_values, logits, next_token_logits, log_probs, nlls
        # gc.collect()
        # torch.cuda.empty_cache()

Per step NLL

In [ ]:
class MinNLLSearcher(object):
    def __init__(self, model, max_depth: int = 31 * 30 + 1, brunch_factor: int = 4, multiple: float = 1.25):
        """Initialize the searcher with a pre-trained model."""

        self.max_depth = max_depth
        self.model = model
        self.best_paths = []
        self.nlls = []
        self.min_nll = float('inf')
        self.brunch_factor = brunch_factor
        self.multiple = multiple

    @staticmethod
    def check_equal_line_lengths(tensor):
        """ Check if all lines in a torch tensor have the same length when called at a line break position."""
        tensor = tensor[0]
        if tensor[-1].item() != 12: # only check if the last token is a line break
            return True
        idx = (tensor == 12).nonzero(as_tuple=True)[0]
        if len(idx) <= 1:
            return True
        return len(torch.unique(idx[1:] - idx[:-1])) == 1
        
    @torch.no_grad()
    def dfs_generate(self, current_ids, current_nll = 0, past_key_values = None, current_depth = 1):
        """Performs Depth-First Search to find the lowest NLL completion."""
        # current_ids is torch.Tensor of Shape: (1, seq_len)
        model = self.model
        max_depth = self.max_depth
        device = model.device
        # Safety check for recursion depth
        if current_depth > max_depth:
            return

        # Prepare inputs for the model
        if past_key_values is None:
            # First call, process the whole sequence
            input_ids_step = current_ids
        else:
            # Subsequent calls, only process the last token
            input_ids_step = current_ids[:, -1:]

        # Get logits and new KV cache
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            outputs = model(
                input_ids=input_ids_step,
                past_key_values=past_key_values,
                use_cache=True, # Crucial for efficiency
            )

        logits = outputs.logits.float() # Shape: (1, input_seq_len, vocab_size)
        new_past_key_values = outputs.past_key_values # Updated KV cache

        # Get logits for the *next* token prediction
        # Logits shape is (batch_size, sequence_length, vocab_size)
        next_token_logits = logits[:, -1, :]

        # Calculate log probabilities and negative log likelihoods
        log_probs = torch.log_softmax(next_token_logits, dim=-1)
        nlls = -log_probs # Shape: (1, vocab_size)

        # Sort potential next tokens by NLL (ascending)
        # We only need to explore the top `branching_factor` candidates
        sorted_nlls, sorted_indices = torch.sort(nlls.squeeze(), descending=False)
        sorted_nlls = sorted_nlls[:self.brunch_factor]
        sorted_indices = sorted_indices[:self.brunch_factor]
        
        # Iterate through the most promising next tokens
        for next_token_id, next_token_nll in zip(sorted_indices, sorted_nlls):
            next_token_id = next_token_id.item()
            next_token_nll = next_token_nll.item()

            potential_total_nll = current_nll + next_token_nll

            # --- Pruning ---
            if potential_total_nll/current_depth >= self.min_nll * self.multiple:
                # If the current path's NLL is already worse than the best complete
                # sequence found so far, prune this branch.
                break

            
            if past_key_values is None: # first call
                next_ids = torch.tensor([[next_token_id]], dtype=torch.long, device=device)
            else: # Append the chosen token
                next_ids = torch.cat(
                    [current_ids, torch.tensor([[next_token_id]], dtype=torch.long, device=device)],
                    dim=1
                )
            
            if not self.check_equal_line_lengths(next_ids):
                # If the line lengths are not equal, prune this branch
                continue
            
            # --- Base Case: EOS token ---
            if next_token_id == 14 and next_ids[0][-2].item() == 12: # Line break followed by EOS
                print(f"Found EOS. Path NLL: {potential_total_nll/current_depth:.4f} | Path Len: {current_depth}")
                self.best_paths.append(next_ids[0].tolist())
                self.nlls.append(potential_total_nll/current_depth)
                self.min_nll = min(self.min_nll, potential_total_nll/current_depth)
                # Continue searching other branches even after finding an EOS
                continue # Don't recurse further down this path

            # --- Recursive Step ---
            # Pass the `new_past_key_values` which contains the cache state *after*
            self.dfs_generate(current_ids=next_ids,
                              current_nll=potential_total_nll,
                              past_key_values=new_past_key_values,
                              current_depth=current_depth + 1,
                             )
        # del new_past_key_values, past_key_values, logits, next_token_logits, log_probs, nlls
        # gc.collect()
        # torch.cuda.empty_cache()